In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os

import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

import torch
import pickle

from tqdm import tqdm

from collections import Counter

from linearization.analyzer import SAELinearizer
from linearization.visualization import visualize_topk

In [7]:
EXPANSION_FACTOR = 32

In [12]:
OVERRIDE = True

path = f"../data/pickled_linearizers/gelu_2l_{EXPANSION_FACTOR}.pkl"

# # Get SAE names automatically
# prefix = "../scripts/checkpoints/64x_resid_mid/"
# sae_names = []
# for i in range(12):
#     with os.scandir(prefix + f"layer{i}") as it:
#         for entry in it:
#             if entry.name.startswith("final") and not entry.name.endswith("sparsity.pt") and entry.is_file():
#                 sae_names.append(entry.path)
# print("SAE names:", sae_names)

# sae_names = [
#     f"../scripts/checkpoints/{EXPANSION_FACTOR}x_gelu_2l/layer0/seed42_ln2_normalized/final_sparse_autoencoder_gelu-2l_blocks.0.ln2.hook_normalized_16384.pt",
#     f"../scripts/checkpoints/{EXPANSION_FACTOR}x_gelu_2l/layer0/seed42_mlp_out/final_sparse_autoencoder_gelu-2l_blocks.0.hook_mlp_out_16384.pt",
#     f"../scripts/checkpoints/{EXPANSION_FACTOR}x_gelu_2l/layer0/seed42_transcoder/final_sparse_autoencoder_gelu-2l_blocks.0.ln2.hook_normalized_16384.pt",
#     f"../scripts/checkpoints/{EXPANSION_FACTOR}x_gelu_2l/layer1/seed42_ln2_normalized/final_sparse_autoencoder_gelu-2l_blocks.1.ln2.hook_normalized_16384.pt",
#     f"../scripts/checkpoints/{EXPANSION_FACTOR}x_gelu_2l/layer1/seed42_mlp_out/final_sparse_autoencoder_gelu-2l_blocks.1.hook_mlp_out_16384.pt",
#     f"../scripts/checkpoints/{EXPANSION_FACTOR}x_gelu_2l/layer1/seed42_transcoder/final_sparse_autoencoder_gelu-2l_blocks.1.ln2.hook_normalized_16384.pt",
#     f"../scripts/checkpoints/{EXPANSION_FACTOR}x_gelu_2l/layer0/seed43_ln2_normalized/final_sparse_autoencoder_gelu-2l_blocks.0.ln2.hook_normalized_16384.pt",
#     f"../scripts/checkpoints/{EXPANSION_FACTOR}x_gelu_2l/layer0/seed43_mlp_out/final_sparse_autoencoder_gelu-2l_blocks.0.hook_mlp_out_16384.pt",
#     f"../scripts/checkpoints/{EXPANSION_FACTOR}x_gelu_2l/layer0/seed43_transcoder/final_sparse_autoencoder_gelu-2l_blocks.0.ln2.hook_normalized_16384.pt",
#     f"../scripts/checkpoints/{EXPANSION_FACTOR}x_gelu_2l/layer1/seed43_ln2_normalized/final_sparse_autoencoder_gelu-2l_blocks.1.ln2.hook_normalized_16384.pt",
#     f"../scripts/checkpoints/{EXPANSION_FACTOR}x_gelu_2l/layer1/seed43_mlp_out/final_sparse_autoencoder_gelu-2l_blocks.1.hook_mlp_out_16384.pt",
#     f"../scripts/checkpoints/{EXPANSION_FACTOR}x_gelu_2l/layer1/seed43_transcoder/final_sparse_autoencoder_gelu-2l_blocks.1.ln2.hook_normalized_16384.pt",
# ]

# Automated SAE name generation
seeds = [42, 43]
layers = [0, 1]
names1 = ["ln2_normalized", "mlp_out", "transcoder"]
names2 = ["ln2.hook_normalized", "hook_mlp_out", "ln2.hook_normalized"]

sae_names = []
for seed in seeds:
    for layer in layers:
        for name1, name2 in zip(names1, names2):
            sae_names.append(
                f"../scripts/checkpoints/{EXPANSION_FACTOR}x_gelu_2l/layer{layer}/seed{seed}_{name1}/final_sparse_autoencoder_gelu-2l_blocks.{layer}.{name2}_16384.pt"
            )

if os.path.exists(path) and not OVERRIDE:
    lin = pickle.load(open(path, "rb"))
else:
    lin = SAELinearizer(
        model_name="gelu-2l",
        sae_names=sae_names,
        layers=[0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1],
        dataset_name="NeelNanda/pile-10k",
        act_name="normalized",
        num_batches=10,
        half_precision=False,
        # transcoder=True,
        # use_gpt=True,
        # dict_mult=64,
        # hook_point="resid_mid",
        run_analysis=True,
    )
    with open(path, "wb") as f:
        pickle.dump(lin, f)

Loaded pretrained model gelu-2l into HookedTransformer
Moving model to device:  cuda
Model device: cuda:0
Tokens shape: torch.Size([125337, 128]), dtype: torch.int64, device: cuda:0
Loading ../scripts/checkpoints/32x_gelu_2l/layer0/seed42_ln2_normalized/final_sparse_autoencoder_gelu-2l_blocks.0.ln2.hook_normalized_16384.pt from disk
Encoder device: cuda:0
Loading ../scripts/checkpoints/32x_gelu_2l/layer0/seed42_mlp_out/final_sparse_autoencoder_gelu-2l_blocks.0.hook_mlp_out_16384.pt from disk
Encoder device: cuda:0
Loading ../scripts/checkpoints/32x_gelu_2l/layer0/seed42_transcoder/final_sparse_autoencoder_gelu-2l_blocks.0.ln2.hook_normalized_16384.pt from disk
Encoder device: cuda:0
Loading ../scripts/checkpoints/32x_gelu_2l/layer1/seed42_ln2_normalized/final_sparse_autoencoder_gelu-2l_blocks.1.ln2.hook_normalized_16384.pt from disk
Encoder device: cuda:0
Loading ../scripts/checkpoints/32x_gelu_2l/layer1/seed42_mlp_out/final_sparse_autoencoder_gelu-2l_blocks.1.hook_mlp_out_16384.pt fro

100%|██████████| 10/10 [00:00<00:00, 28.10it/s]


Num dead 6.103515625e-05


100%|██████████| 10/10 [00:00<00:00, 25.74it/s]


Num dead 0.02044677734375


...

100%|██████████| 10/10 [00:00<00:00, 24.74it/s]


Num dead 0.38287353515625


100%|██████████| 10/10 [00:00<00:00, 23.80it/s]


Num dead 0.0487060546875


100%|██████████| 10/10 [01:06<00:00,  6.68s/it]


In [22]:
# 5 features, uniformly sampled by frequency, seed 42 layer 0:

sae_name = "../scripts/checkpoints/32x_gelu_2l/layer0/seed42_transcoder/final_sparse_autoencoder_gelu-2l_blocks.0.ln2.hook_normalized_16384.pt"
freqs = lin.frequencies[sae_name]

my_sample = np.argsort(freqs.cpu().numpy())[::-1][::len(freqs) // 5]
print(len(my_sample))

6


In [24]:
lin.set_feature(my_sample[0], sae_name)

 16%|█▌        | 4/25 [00:00<00:01, 14.09it/s]


Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "/home/phil/miniconda3/envs/mats/lib/python3.10/site-packages/IPython/core/interactiveshell.py", line 3526, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/tmp/ipykernel_644995/3163856484.py", line 1, in <module>
    lin.set_feature(my_sample[0], sae_name)
  File "/home/phil/mlp_linearization/src/linearization/analyzer.py", line 110, in set_feature
    self.top_examples = top_activating_examples(**self._kw2, num_batches=num_batches)
  File "/home/phil/mlp_linearization/src/linearization/analyses/feature.py", line 84, in top_activating_examples
    activations, tokens = _get_sample(model, sae, data, feature_idx, layer, act_name, num_batches)
  File "/home/phil/mlp_linearization/src/linearization/analyses/feature.py", line 25, in _get_sample
    _, cache = model.run_with_cache(tokens, names_filter=get_act_name(act_name, layer, layer_name))
  File "/home/phil/miniconda3/envs/mats/lib/python3.10/site-packages/transformer_